Для выполнения нескольких одинаковых тестов с различными параметрами принято использовать метод `subTest()`.

Все данные, на которых будет проводиться тестирование, собирают в итерируемый объект (например, в словарь, кортеж или список), проходят по нему циклом — и на каждой итерации передают очередную «порцию» тестовых данных в тест.

В теле цикла прописывается контекстный менеджер `with self.subTest()`, а в теле контекстного менеджера — проверяемые утверждения. 

***
## Циклом по тестам

Лучше всего увидеть работу метода `subTest()` в живом коде.


In [ ]:
from unittest import TestCase


def get_square(num):
    """Возвращает квадрат полученного аргумента"""
    return num ** 2


class TestExample(TestCase):

    def test_square(self):
        """Тест возведения в квадрат."""
        # Проверим три утверждения: при возведении первого числа в квадрат
        # функция вернёт второе число.
        # Исходные данные соберём в кортеж, содержащий в себе другие кортежи.
        values_results = (
            (2, 4),   # С этими параметрами тест вернёт OK.
            (3, 10),  # С этими параметрами тест провалится.
            (4, 20),  # И с этими параметрами - тоже провалится.
        )
        # Цикл, в котором кортежи, вложенные в values_results, 
        # распаковываются в переменные value и expected_result:
        for value, expected_result in values_results:
            # subTest в качестве контекстного менеджера.
            with self.subTest():
                result = get_square(value)
                # Тестовое утверждение, которое будет вызвано несколько раз
                # с разными значениями переменных.
                self.assertEqual(result, expected_result) 


Распаковку вложенных кортежей не обязательно производить в той же строке, где объявляется цикл, распаковать можно и внутри цикла:


In [ ]:
from unittest import TestCase


def get_square(num):
    return num ** 2


class TestExample(TestCase):

    def test_square(self):
        """Тест возведения в квадрат."""
        values_results = (
            (2, 4),
            (3, 10),
            (4, 20),
        )
        # В цикле присваиваем кортежи переменной item.
        for item in values_results:
            # Отдельной строкой распаковываем item на две переменных.
            value, expected_result = item
            # Остальной код без изменений.
            with self.subTest():
                result = get_square(value)
                self.assertEqual(result, expected_result) 

Когда во вложенных кортежах содержится много элементов — такой вариант будет более читаемым.

Обратите внимание, в выводе написано, что был запущен один тест, но падал он дважды. Никакого противоречия: «тест» — это единственный метод класса `TestExample()`. А падения теста вызваны проверками в `subTest()` — и их было несколько.


In [ ]:
======================================================================
FAIL: test_square (test_subtest.TestExample) (<subtest>)
Тест возведения в квадрат.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<путь до файла>", line 15, in test_square
    self.assertEqual(value ** 2, result)
AssertionError: 9 != 10

======================================================================
FAIL: test_square (test_subtest.TestExample) (<subtest>)
Тест возведения в квадрат.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<путь до файла>", line 15, in test_square
    self.assertEqual(value ** 2, result)
AssertionError: 16 != 20

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (failures=2) 

В результатах тестирования отображается и докстринг, в котором описано назначение теста. Когда тестов мало — в докстринге нет необходимости, но когда количество тестов вырастет — докстринги помогут ориентироваться в результатах.

Информативность тестов можно улучшить не только докстрингами: в сообщении об ошибке можно выводить аргументы, на которых упал тест. Для этого в метод `subTest()` передают именованные аргументы со значениями, на которых проводится тестирование. 

Передадим в `subTest()` именованный аргумент со значением из распакованного кортежа:


In [ ]:
...
with self.subTest(value=value):
    ... 

При падении теста этот аргумент и его значение отобразятся в консоли:


In [ ]:
FAIL: test_square (test_subtest.TestExample) (value=4) 

В `subTest()` можно передать несколько аргументов:


In [ ]:
...
with self.subTest(value=value, expected_result=expected_result):
    ... 

При падении теста все они появятся в выводе:


In [ ]:
FAIL: test_square (test_subtest.TestExample) (value=4, expected_result=20) 

В именованном аргументе `msg` можно передать в `subTest()` сообщение об ошибке:


In [ ]:
...
with self.subTest(
        value=value, 
        expected_result=expected_result,
        msg=f'Возведение числа {value} в квадрат дало результат,\n'
            f'отличающийся от ожидаемого {expected_result}',
        # Символы \n в конце первой строки сообщения обозначают перенос строки:
        # текст, следующий за этими символами, 
        # в консоли будет перенесён на новую строчку.
):
    ... 

В консоль будет выведено


In [ ]:
FAIL: test_square (post-tests.test_1.TestExample) [Возведение числа 4 в квадрат дало результат,
отличающийся от ожидаемого 20] (value=4, expected_result=20) 

Текст сообщения можно передать первым позиционным аргументом:


In [ ]:
...
with self.subTest(
        f'Возведение числа {value} в квадрат дало результат,\n'
        f'отличающийся от ожидаемого {expected_result}',
        value=value, 
        expected_result=expected_result,
):
    ... 

Вывод в консоль будет таким же, как в примере выше.

Набор данных для тестирования хранят в итерируемых объектах (в приведённом примере это кортеж `values_results`, содержащий вложенные кортежи). Технически тип этого объекта может быть любым, единых стандартов на этот счёт нет; но у кортежей есть ряд преимуществ перед остальными типами данных:

1. Элементы кортежа могут быть неуникальными.

2. Кортежи требуют меньше оперативной памяти для хранения данных (по сравнению со списками).

3. Для распаковки кортежа в цикле не требуются дополнительные методы (как например, `items()` для распаковки словарей).